In [1]:
import bibtexparser

In [105]:
import requests
import json

#https://levelup.gitconnected.com/writing-a-python-reference-web-scraper-with-rapidapi-5e810e0e1d88
RAPID_API_KEY = "d9e421fd33mshbe21a72ba0d3dfep1606ddjsnc9a4db7d1dab"

def create_request_url(title):
    q_string = title.replace(' ', '+')
    return f"https://google-search3.p.rapidapi.com/api/v1/search/q={q_string}"
    
def get_request_data(i, title):
    """Retrieves a link for a given title from the references.
            
            - if no link is found: return 'no link found'
            - if error on req:     return 'request failed'
    """
    
    headers = {
        'x-user-agent': "desktop",
        'x-proxy-location': "EU",
        'x-rapidapi-host': "google-search3.p.rapidapi.com",
        'x-rapidapi-key': RAPID_API_KEY
    }
    
    query_s = create_request_url(title)
    
    link = ""

    # if you want a verbose output for each link, uncomment the print statements
    # print(f"Getting link for [{i}]: {title[:20]}...")
    
    try:
        r = requests.request("GET", query_s, headers=headers)
    except ConnectionError:
        pass
    
    #print(query_s)
    #print(r)

    if r.status_code == 200:
        j = json.loads(r.text)
        try:
            link += j['results'][0]['link']
        except:
            link += 'no link found'
    else:
        link += 'request failed'

    # print(f"Done: [{link}]")

    return link,r

In [156]:
with open('bib_google.bib') as bibtex_file:
    bib_g = bibtexparser.load(bibtex_file).entries

with open('bib_orcid.bib') as bibtex_file:
    bib_o = bibtexparser.load(bibtex_file).entries

In [171]:
#sort google scholar by year
years = [ int(paper['year']) for paper in bib_g]
bib_g=sorted(bib_g, key=lambda d: int(d['year']),reverse=True) 


In [172]:
yml_string=""
for i,paper in enumerate(bib_g):
    #print(paper)
    print(f"Processing [{i}]  {paper['title']}......")
    
    #search doi from orcid bib
    doi = "NA"
    for orcid in bib_o:
        a= paper['title'].lower().strip().split()
        b= orcid['title'].lower().strip().split()
        similarity = set( a ) & set( b )
        if( len(a) - len(similarity) <3 and 'doi' in orcid):
            #print(similarity)
            doi = orcid['doi']
            break
    
    yml_string+=f"- authors: \"{paper['author']}\" " + "\n"
    yml_string+=f"  title: \"{paper['title']}\" " + "\n"
    
    if('journal' in paper): source = paper['journal']
    elif('publisher' in paper): source = paper['publisher']
    else: source = "Preprint"
    yml_string+=f"  journal: \"{source}\" "  + "\n"
    yml_string+=f"  pubdate:\n"
    yml_string+=f"    year: {int(paper['year'])}\n"

    if ('volume' not in paper): paper['volume']="10"
    vip = f"{int(paper['volume'])}"
    if('  pages' in paper): vip+=f":{paper['pages']}"
    yml_string+=f"  vip: \"{vip}\"\n"
    yml_string+=f"  doi: {doi}\n"
    yml_string+=f"  link:\n"
    link = f"http://doi.org/{doi}"
    yml_string+=f"    url: {link}\n" 
    yml_string+=f"    display: {source}, {vip}\n"

    yml_string+=f"  image: default.png\n"
    yml_string+=f"  marked: 0\n"
    yml_string+=f"  highlight: 0\n"
    yml_string+=f"  news2:\n"

    yml_string+=f"\n"


with open("publist.yml", "w") as text_file:
    text_file.write(yml_string)

Processing [0]  An GPU-accelerated particle tracking method for Eulerian--Lagrangian simulations using hardware ray tracing cores......
Processing [1]  Digital quantification of fracture in full-scale rock using micro-CT images: A fracturing experiment with N2 and CO2......
Processing [2]  Accelerating linear solvers for Stokes problems with C++ metaprogramming......
Processing [3]  Viscosity robust weak Galerkin finite element methods for Stokes problems......
Processing [4]  Streamline-based simulation of nanoparticle transport in field-scale heterogeneous subsurface systems......
Processing [5]  An Open-Source Code for Fluid Flow Simulations in Unconventional Fractured Reservoirs......
Processing [6]  Full-Sample X-ray Microcomputed Tomography Analysis of Supercritical CO2 Fracturing in Tight Sandstone: Effect of Stress on Fracture Dynamics......
Processing [7]  Advanced modelling with the MATLAB reservoir simulation toolbox......
Processing [8]  Implementation of streamline simulat

In [97]:
paper

{'journal': 'Geosciences',
 'title': 'An Open-Source Code for Fluid Flow Simulations in Unconventional Fractured Reservoirs',
 'author': 'Bin Wang and Corrado Fidelibus',
 'pages': '106',
 'number': '2',
 'volume': '11',
 'publisher': '{MDPI} {AG}',
 'month': 'feb',
 'year': '2021',
 'url': 'https://doi.org/10.3390%2Fgeosciences11020106',
 'doi': '10.3390/geosciences11020106',
 'ENTRYTYPE': 'article',
 'ID': 'Wang_2021'}